In [ ]:
import pandas as pd
import os
import fnmatch
import numpy as np

In [ ]:
file_path_index =pd.read_csv("file_path_index.csv")
merge_all=pd.read_csv('merge_all.csv')

d_index = merge_all[merge_all['BG'].isnull()].index
merge_all=merge_all.drop(d_index)

x = list(np.arange(0, 400, 1))
features=['person_time_idx','Person No','time','Time','count_time','count','BG','Area','FW_25','FW_50','FW_75','FW_100','HR']
x=features+x
output_df=pd.DataFrame(columns=x)


In [ ]:
def get_vally(minmax_df):
    vallys_tStamp=minmax_df.loc[minmax_df['M/m']=='lmin']['t(sec)']
    return vallys_tStamp
def get_pulse_sig(vallys_tStamp):
    global output_df
    signal_df=pd.read_csv(file_path_index.loc[file_path_index['person_time_idx']=='001_0903']['R4signal'].values[0],delim_whitespace=True,skiprows=6)
    return signal_df['AC']

def get_profile(pid_t_idx,pid):
    profile=list(merge_all.loc[merge_all['person_time_idx']==pid_t_idx,['Time','count_time','count','BG']].values[0])
    return profile

def FWnM(pulse, peak_amp, peak_i, n=[0.5]):
    #FWnM = []
    total_len = len(pulse)
    for ni in n:
        thrd = ni*peak_amp + pulse[0]
        left_i = peak_i-len(pulse[:peak_i][pulse[:peak_i]<=thrd])
        right_i = len(pulse[peak_i:][pulse[peak_i:]>=thrd])
        FWnM = round((right_i+left_i)*4,2) # 4ms
        #FWnM.append((right_i+left_i)*100/total_len) # %
    
    return FWnM

In [ ]:
#Main flow func
total_sample=merge_all['person_time_idx'].shape[0]
z=0
time=0
tt=0
output_lst=[]
for pid_t_idx in merge_all['person_time_idx']:
    z=z+1
    if z%537==0:
        print(z/537*10, " % done")
    
    try:
        pid=int(pid_t_idx.split('_')[0])
        if(pid!=tt):
            time=1
            profile_lst = [pid_t_idx,pid,time]+get_profile(pid_t_idx,pid)
            tt=pid
        else:
            time=time+1
            profile_lst = [pid_t_idx,pid,time]+get_profile(pid_t_idx,pid)
        #profile_lst = [pid_t_idx,pid]+get_profile(pid_t_idx,pid)
        #print(pid_t_idx)

        #import file
        minmax_df=pd.read_csv(file_path_index.loc[file_path_index['person_time_idx']==pid_t_idx]['R4minmax'].values[0],delim_whitespace=True)
        signal_df=pd.read_csv(file_path_index.loc[file_path_index['person_time_idx']==pid_t_idx]['R4signal'].values[0],delim_whitespace=True,skiprows=6)

    
        # get vally_t, vally_idx , HR
        vallys_tStamp=minmax_df.loc[(minmax_df['M/m']=='lmin')&(minmax_df['Sig']=='IR')]['t(sec)']
        HR=[60/(float(vallys_tStamp[(i+1)])-float(vallys_tStamp[i])) for i in range(len(vallys_tStamp)-1)]
        vallys_tStamp=vallys_tStamp.drop(vallys_tStamp.index[0])
        
        vallys_min_index=minmax_df.loc[(minmax_df['M/m']=='lmin')&(minmax_df['Sig']=='IR')]['index'].apply(int)
        vallys_min_index=vallys_min_index.drop(vallys_min_index.index[0])
        
        vallys_max1_index=minmax_df.loc[(minmax_df['M/m']=='lmax')&(minmax_df['Sig']=='IR')]['index'].apply(int)
        vallys_max1_index=vallys_max1_index.drop(vallys_max1_index.index[0])
        
        vallys_max2_index=minmax_df.loc[(minmax_df['M/m']=='lmx2')&(minmax_df['Sig']=='IR')]['index'].apply(int)
        vallys_max2_index=vallys_max2_index.drop(vallys_max2_index.index[0])
        
        IR_max = [] 
        for i1,i2 in zip(vallys_max1_index.values, vallys_max2_index.values):
            if signal_df['AC'].loc[i1]>signal_df['AC'].loc[i2]:
                IR_max.append(i1)
            else:
                IR_max.append(i2)
        start_idx = vallys_min_index.iloc[:-1].values
        end_idx = vallys_min_index.iloc[1:].values
        peak_idx = IR_max[:-1]
        
        
        if (len(vallys_tStamp)!=len(HR)):
            print(len(vallys_tStamp),len(vallys_index),HR)
    except:
        print('Error on : ',pid_t_idx)
    
    count=1
    
    #for index,hr in zip(vallys_min_index,HR):
    
    for (si,ei,pi,hr) in zip(start_idx, end_idx, peak_idx,HR):
        if ((int(si)< 400)|(int(si)+1 > len(signal_df))):
            continue
        temp_lst= profile_lst.copy()
        #print("profile_lst len :",len(profile_lst),"content :",profile_lst)
        
        p_ref_idx = pi-si
        rise_area, fall_area = 0, 0
        y0 = signal_df['AC'].loc[si]
        amp = signal_df['AC'].loc[pi]-y0
                
        half_level = amp/2 + y0
        hli, hri = 0, 0
        # inside pulse loop
        pulse_seg = signal_df.loc[si+1:ei]['AC']
        for i,val in enumerate(pulse_seg,1):
            if i<p_ref_idx:
                rise_area += (val-y0)
                #if val<=half_level:
                #    hli = i
            else:
                fall_area += (val-y0)
                #if val>=half_level:
                #    hri = i
        #portion = [0.25, 0.5, 0.75]
        
        temp_lst.append((rise_area+fall_area))
        
        fwhm_dic_25 = FWnM(pulse_seg.values, amp, p_ref_idx, n=[0.25])
        temp_lst.append(fwhm_dic_25)
        fwhm_dic_50 = FWnM(pulse_seg.values, amp, p_ref_idx, n=[0.5])
        temp_lst.append(fwhm_dic_50)
        fwhm_dic_75 = FWnM(pulse_seg.values, amp, p_ref_idx, n=[0.75])
        temp_lst.append(fwhm_dic_75)
        fwhm_dic_100 = (ei-si)*4
        temp_lst.append(fwhm_dic_100)
        
        temp_lst.append(hr)
        
        #add 200pulse point to data_entry
        temp_lst=temp_lst+list(signal_df.iloc[(int(si)-399):(int(si)+1)]['AC'])
        
        output_lst.append(temp_lst)
        count=count+1

In [ ]:
output_df=pd.DataFrame(output_lst,columns=x)
print(output_df.shape)
output_df.head()

In [ ]:
BG_last=[]
HR_last=[]
Area_last=[]
FW_25_last=[]
FW_50_last=[]
FW_75_last=[]
FW_100_last=[]
ppg_last=[]


for i,item in enumerate(output_df['Person No']):
    p=output_df['count'].iloc[i]
    if(p==0):
        index = np.random.choice(output_df.index)
        ppg_last.append(output_df.iloc[index][13:413].values)
        BG_last.append(output_df.iloc[index]['BG'])
        HR_last.append(output_df.iloc[index]['HR'])
        Area_last.append(output_df.iloc[index]['Area'])
        FW_25_last.append(output_df.iloc[index]['FW_25'])
        FW_50_last.append(output_df.iloc[index]['FW_50'])
        FW_75_last.append(output_df.iloc[index]['FW_75'])
        FW_100_last.append(output_df.iloc[index]['FW_100'])
    else:
        if(output_df['count_time'].iloc[i]==2):
            tmp=output_df[(output_df['Person No']==p)]
            if(tmp.empty):
                index = np.random.choice(output_df.index)
                ppg_last.append(output_df.iloc[index][13:413].values)
                BG_last.append(output_df.iloc[index]['BG'])
                HR_last.append(output_df.iloc[index]['HR'])
                Area_last.append(output_df.iloc[index]['Area'])
                FW_25_last.append(output_df.iloc[index]['FW_25'])
                FW_50_last.append(output_df.iloc[index]['FW_50'])
                FW_75_last.append(output_df.iloc[index]['FW_75'])
                FW_100_last.append(output_df.iloc[index]['FW_100'])
            else:
                index = np.random.choice(tmp.index)
                ppg_last.append(output_df.iloc[index][13:413].values)
                BG_last.append(output_df.iloc[index]['BG'])
                HR_last.append(output_df.iloc[index]['HR'])
                Area_last.append(output_df.iloc[index]['Area'])
                FW_25_last.append(output_df.iloc[index]['FW_25'])
                FW_50_last.append(output_df.iloc[index]['FW_50'])
                FW_75_last.append(output_df.iloc[index]['FW_75'])
                FW_100_last.append(output_df.iloc[index]['FW_100'])
        else:
            tmp=output_df[(output_df['count']==p)&(output_df['count_time']==output_df['count_time'].iloc[i]-1)]
            if(tmp.empty):
                index = np.random.choice(output_df.index)
                ppg_last.append(output_df.iloc[index][13:413].values)
                BG_last.append(output_df.iloc[index]['BG'])
                HR_last.append(output_df.iloc[index]['HR'])
                Area_last.append(output_df.iloc[index]['Area'])
                FW_25_last.append(output_df.iloc[index]['FW_25'])
                FW_50_last.append(output_df.iloc[index]['FW_50'])
                FW_75_last.append(output_df.iloc[index]['FW_75'])
                FW_100_last.append(output_df.iloc[index]['FW_100'])
            else:
                index = np.random.choice(tmp.index)
                ppg_last.append(output_df.iloc[index][13:413].values)
                BG_last.append(output_df.iloc[index]['BG'])
                HR_last.append(output_df.iloc[index]['HR'])
                Area_last.append(output_df.iloc[index]['Area'])
                FW_25_last.append(output_df.iloc[index]['FW_25'])
                FW_50_last.append(output_df.iloc[index]['FW_50'])
                FW_75_last.append(output_df.iloc[index]['FW_75'])
                FW_100_last.append(output_df.iloc[index]['FW_100'])

In [ ]:
arr1 = np.array(ppg_last)
arr1.shape

In [ ]:
dataframe = pd.DataFrame(arr1)

In [ ]:
list1 = list(np.arange(400)+400)
list2 = []
for item in list1:
    list2.append(str(item))

In [ ]:
dataframe.columns=list2

In [ ]:
output_df['BG_last']=BG_last
output_df['Area_last']=Area_last
output_df['FW_25_last']=FW_25_last
output_df['FW_50_last']=FW_50_last
output_df['FW_75_last']=FW_75_last
output_df['FW_100_last']=FW_100_last
output_df['HR_last']=HR_last

end_df = pd.concat([output_df, dataframe], axis=1)

In [ ]:
end_df.to_csv("Signal_pair.csv")

In [ ]:
end_df.dtypes